In [2]:
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
import sagemaker
role = sagemaker.get_execution_role()
from sagemaker.tuner import IntegerParameter, ContinuousParameter
from sagemaker.tuner import HyperparameterTuner

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [ ]:
estimator = TensorFlow(
    entry_point='train.py',
    source_dir='scripts',
    role=role,
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    framework_version='2.13',
    py_version='py310',
    script_mode=True,
    hyperparameters={
        'epochs': 50,
        'batch_size': 32,
        'learning_rate': 0.001
    },
    output_path=f's3://project-artist/output'
)

hyperparameter_ranges = {
    'conv_filters': IntegerParameter(32, 128),
    'dense_units': IntegerParameter(256, 1024),
    'dropout_rate': ContinuousParameter(0.2, 0.5),
    'num_conv_layers': IntegerParameter(2, 4),
    'epochs': IntegerParameter(10, 50),
    'batch_size': IntegerParameter(16, 64),
    'learning_rate': ContinuousParameter(0.0001, 0.01)
}

objective_metric_name = 'val_accuracy'
objective_type = 'Maximize'

metric_definitions = [{'Name': 'val_accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name='val_accuracy',
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_jobs=20,
    max_parallel_jobs=3,
    objective_type='Maximize'
)

input_data = TrainingInput('s3://project-artist/data3.zip', content_type='application/zip')
tuner.fit({'training': input_data})


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


....

In [8]:
from sagemaker.tuner import HyperparameterTuner
tuner = HyperparameterTuner.attach('tensorflow-training-250919-1629')
tuner.analytics().dataframe()

,batch_size,conv_filters,dense_units,dropout_rate,epochs,learning_rate,num_conv_layers,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,44.0,128.0,840.0,0.481085,10.0,0.000100,4.0,tensorflow-training-250919-1629-021-28b63aa9,Completed,0.778198,2025-09-21 04:34:37+00:00,2025-09-21 14:31:42+00:00,35825.0
1,42.0,77.0,988.0,0.409637,12.0,0.000110,4.0,tensorflow-training-250919-1629-020-2db81c12,Completed,0.770977,2025-09-21 03:52:19+00:00,2025-09-21 08:52:40+00:00,18021.0
2,23.0,88.0,415.0,0.464058,12.0,0.000106,4.0,tensorflow-training-250919-1629-019-1b33e4d1,Completed,0.786107,2025-09-21 02:20:41+00:00,2025-09-21 08:03:29+00:00,20568.0
3,48.0,83.0,579.0,0.468575,12.0,0.000100,4.0,tensorflow-training-250919-1629-018-60de24f0,Completed,0.786795,2025-09-20 22:18:36+00:00,2025-09-21 03:44:39+00:00,19563.0
4,56.0,89.0,532.0,0.339922,14.0,0.000256,4.0,tensorflow-training-250919-1629-017-1682f9c6,Completed,0.741403,2025-09-20 21:25:57+00:00,2025-09-21 04:34:13+00:00,25696.0
5,36.0,84.0,833.0,0.239563,14.0,0.000200,4.0,tensorflow-training-250919-1629-016-9edff400,Completed,0.766162,2025-09-20 19:45:13+00:00,2025-09-21 02:19:20+00:00,23647.0
6,59.0,88.0,326.0,0.359433,13.0,0.000128,3.0,tensorflow-training-250919-1629-015-f7b24599,Completed,0.759629,2025-09-20 16:43:33+00:00,2025-09-20 21:22:07+00:00,16714.0
7,26.0,87.0,317.0,0.228267,13.0,0.000176,4.0,tensorflow-training-250919-1629-014-9d526e35,Completed,0.773384,2025-09-20 15:52:28+00:00,2025-09-20 22:17:12+00:00,23084.0
8,16.0,109.0,803.0,0.229173,10.0,0.000100,3.0,tensorflow-training-250919-1629-013-dae44f3a,Completed,0.725928,2025-09-20 11:40:02+00:00,2025-09-20 19:38:44+00:00,28722.0
9,58.0,91.0,860.0,0.234592,15.0,0.000100,4.0,tensorflow-training-250919-1629-012-370d11ff,Completed,0.765131,2025-09-20 08:18:10+00:00,2025-09-20 16:37:26+00:00,29956.0


In [10]:
best_estimator = tuner.best_estimator()
print(best_estimator.hyperparameters())


2025-09-21 03:44:41 Starting - Found matching resource for reuse
2025-09-21 03:44:41 Downloading - Downloading the training image
2025-09-21 03:44:41 Training - Training image download completed. Training in progress.
2025-09-21 03:44:41 Uploading - Uploading generated training model
2025-09-21 03:44:41 Completed - Resource released due to keep alive period expiry
{'_tuning_objective_metric': '"val_accuracy"', 'batch_size': '48', 'conv_filters': '83', 'dense_units': '579', 'dropout_rate': '0.46857479751620845', 'epochs': '12', 'learning_rate': '0.00010000000000000009', 'num_conv_layers': '4', 'sagemaker_container_log_level': '20', 'sagemaker_estimator_class_name': '"TensorFlow"', 'sagemaker_estimator_module': '"sagemaker.tensorflow.estimator"', 'sagemaker_job_name': '"tensorflow-training-2025-09-19-16-29-31-073"', 'sagemaker_program': '"train.py"', 'sagemaker_region': '"us-east-2"', 'sagemaker_submit_directory': '"s3://my-first-aws-bucket-kamila/tensorflow-training-2025-09-19-16-29-31